#### 🔎 5.5.1 교차 검증을 통한 집값 예측 모델 평가

##### 💡 **모델 평가하기**

- `from sklearn.model_selection import KFold`
    - 학습 데이터 세트를 k개의 부분 데이터 세트(폴드)로 나눈 후
    - k-1개의 폴드는 학습데이터, 1개는 검증 데이터로 사용할 수 있도록
    - **전체 학습 데이터 세트에서 인덱스를 나눠주는 역할** 

In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

import torch
from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt

##### 💡 **데이터 프레임을 넘파이 배열로 만들기**

In [5]:
df = pd.read_csv('C:/nozzi/Pytorch/딥러닝을 위한 파이토치 입문/CH5. 인공 신경망/data/reg.csv', index_col=[0])
X = df.drop('Price', axis=1).to_numpy()
Y = df['Price'].to_numpy().reshape((-1, 1))

##### 💡 **텐서 데이터 만들기**

- trainset는 교차 검증을 위해 나누기 때문에 미리 DataLoader를 정의하지 않는다.

In [9]:
class TensorData(Dataset) :
    def __init__(self, x_data, y_data) :
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index) :
        return self.x_data[index], self.y_data[index]
    
    def __len__(self) :
        return self.len
    
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.7)
trainset = TensorData(X_train, Y_train)
testset = TensorData(X_test, Y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

##### 💡 **모델 구축**

In [10]:
class Regressor(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.fc1 = nn.Linear(13, 50, bias=True)
        self.fc2 = nn.Linear(50, 30, bias=True)
        self.fc3 = nn.Linear(30, 1, bias=True)
        
    def forward(self, x) :
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

##### 💡 **손실 함수와 교차 검증 정의**

In [11]:
kfold = KFold(n_splits=3, shuffle=True)
criterion = nn.MSELoss()

##### 💡 **평가 함수 정의**

In [38]:
def evaluation(dataloader) :
    
    # 예측값/실제값 저장할 빈 텐서 
    predictions = torch.tensor([], dtype=torch.float)
    actual = torch.tensor([], dtype=torch.float)
    
    with torch.no_grad() :
        model.eval()
        for data in dataloader :
            inputs, values = data
            outputs = model(inputs)
            
            # cat을 통해 예측값/실제값 누적
            predictions = torch.cat((predictions, outputs), 0)
            actual = torch.cat((actual, values), 0)
            
    # CPU용 텐서를 넘파이 배열로 변환
    predictions = predictions.numpy()
    actual = actual.numpy()
    rmse = np.sqrt(mean_squared_error(predictions, actual))
    
    # 여기서는 상관없지만, 평가 시에는 정규화 기술을 배제하여 온전한 모델로 평가해야 한다. (.eval() 사용)
    # 즉, 드롭아웃이나 배치 정규화 등과 같이 학습 시에만 사용하는 기술들이 적용된 모델은 평가 시에는 비활성화해야 하며,
    # 다시 .train() 사용해야 한다.
    model.train()
    return rmse

##### 💡 **교차 검증을 이용한 학습 및 평가**

In [27]:
print("trainset 길이 : ",len(trainset))
for fold, (train_idx, val_idx) in enumerate(kfold.split(trainset)) :
    print(fold)
    print(train_idx, len(train_idx))
    print(val_idx, len(val_idx))

trainset 길이 :  151
0
[  0   2   3   4   5   7  10  11  12  13  14  15  16  17  20  21  22  24
  25  28  29  30  31  32  35  36  37  39  40  42  43  44  46  49  53  55
  56  58  59  63  64  67  68  69  70  71  72  74  75  76  79  81  82  83
  85  86  88  92  93  94  95  96  99 101 102 103 104 105 106 109 111 112
 114 115 116 117 118 119 121 122 124 126 127 130 131 132 133 135 136 137
 138 139 140 142 143 144 145 148 149 150] 100
[  1   6   8   9  18  19  23  26  27  33  34  38  41  45  47  48  50  51
  52  54  57  60  61  62  65  66  73  77  78  80  84  87  89  90  91  97
  98 100 107 108 110 113 120 123 125 128 129 134 141 146 147] 51
1
[  1   2   3   6   7   8   9  11  12  16  17  18  19  20  21  23  24  25
  26  27  28  30  31  33  34  37  38  40  41  43  44  45  47  48  49  50
  51  52  54  56  57  60  61  62  65  66  71  73  75  76  77  78  79  80
  82  84  87  89  90  91  93  94  95  96  97  98  99 100 101 102 104 106
 107 108 110 111 113 114 116 118 119 120 121 122 123 124 125 12

In [39]:
# 검증 점수를 산출하기 위해 fold 별 loss 저장 리스트
validation_loss = []

# kfold.split(trainset)을 이용하여 나눠진 학습 데이터의 인덱스를 불러온다. 
for fold, (train_idx, val_idx) in enumerate(kfold.split(trainset)) :
    
    # TensorData로 정의된 데이터의 일부를 불러와 데이터의 일부를 불러와 배치 데이터 형태로 활용할 수 있도록 
        # 길이, 인덱스 사용할 수 있도록 
    # DataLoader, SubsetRandomSampler를 함께 사용한다. 
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, sampler = train_subsampler)
    valloader = torch.utils.data.DataLoader(trainset, batch_size=32, sampler = val_subsampler)
    
    # (fold별로) 매 학습마다 모델 파라미터를 초기화하기 위해 for문 안에 모델을 선언
    model = Regressor()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-7)
    
    for epoch in range(400) :
        for data in trainloader : 
            inputs, values = data
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, values)
            loss.backward()
            optimizer.step()
            
    # 각 검증마다 학습 데이터와 검증 데이터를 가지고 RMSE 계산 (총 3번)
    train_rmse = evaluation(trainloader)
    val_rmse = evaluation(valloader)
    print("k-fold", fold, "Train Loss: %.4f, Validation Loss: %.4f" %(train_rmse, val_rmse))
    
    # 검증 RMSE 저장
    validation_loss.append(val_rmse)           

k-fold 0 Train Loss: 0.1115, Validation Loss: 0.2015
k-fold 1 Train Loss: 0.1171, Validation Loss: 0.1317
k-fold 2 Train Loss: 0.1143, Validation Loss: 0.1387


##### 💡 **검증 점수 산출**

In [41]:
validation_loss = np.array(validation_loss)
mean = np.mean(validation_loss)
std = np.std(validation_loss)
print("validation Score: %4f, +- %.4f" %(mean, std))

validation Score: 0.157294, +- 0.0314


##### 💡 **모델 평가**

- Train, Test의 차이가 줄어들었다.

In [42]:
# 다시 합쳐주기 (학습 데이터 전체에 대해 DataLoader 생성해서 RMSE 계산)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=False)
train_rmse = evaluation(trainloader)
test_rmse = evaluation(testloader)
print("Train RMSE: %.4f" %train_rmse)
print("Test RMSE: %.4f" %test_rmse)

Train RMSE: 0.1229
Test RMSE: 0.1318
